# Aggregate data in PostGIS

```sql
SELECT 
  SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads_velo.geom))) AS velo,
  SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS total
FROM 
  zh_districts, 
  zh_roads,
  zh_roads_velo = (SELECT geom FROM zh_roads WHERE velo=1)
WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
GROUP BY zh_districts;
```

WORK IN PROGRESS

Simple query with airport count:
```sql
SELECT c."NAME" as name, COUNT(a.geometry) AS count_airports 
FROM ne_countries AS c, ne_airports AS a
WHERE st_contains(c.geometry,a.geometry)
GROUP BY c."NAME";
```

Query with airport count and average GDP per airport:
```sql
SELECT 
    c."NE_ID" AS objectid,
    c."NAME" AS country_name, 
    COUNT(a.ne_id) AS count_airports,
    c."GDP_MD" AS gdp_musd,
    ROUND(c."GDP_MD" / COUNT(a.ne_id)) AS gdp_per_airport,
    c.geometry AS geometry
FROM ne_countries AS c, ne_airports AS a
WHERE st_contains(c.geometry,a.geometry)
GROUP BY c."NE_ID", c."NAME", c."GDP_MD", c.geometry;
```

Create a view:
```sql
CREATE or REPLACE VIEW airports_per_country AS
    SELECT 
        c."NE_ID" AS objectid,
        c."NAME" AS country_name, 
        COUNT(a.ne_id) AS count_airports,
        c."GDP_MD" AS gdp_musd,
        ROUND(c."GDP_MD" / COUNT(a.ne_id)) AS gdp_per_airport,
        c.geometry AS geometry
    FROM ne_countries AS c, ne_airports AS a
    WHERE st_contains(c.geometry,a.geometry)
    GROUP BY c."NE_ID", c."NAME", c."GDP_MD", c.geometry;
```